<a href="https://colab.research.google.com/github/JacopoKahl/CycleGAN/blob/main/CycleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CycleGAN

Composed by 4 models: 2 generators and 2 discriminators

**Objective**: Train generators (together with discriminators) to be able to transgorm input images.

**Dataset**: apple2orange image dataset

**Notation**:

1.   **g_AB**: Generator for A->B image conversion
2.   **G_BA**: Generator for B->A image conversion
3.   **d_A**: Discriminator for A type images
4.   **d_B**: Discriminator for B type images



In [17]:
%tensorflow_version 1.x

##Imports


*   Need to install keras-contrib from GitHub!



In [18]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ym61iugl
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ym61iugl
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=16ebfb9c8a926b0ff5a4cc50430524b6c552e19a5bb1847d363534fbbaff5de4
  Stored in directory: /tmp/pip-ephem-wheel-cache-q3hk4ecc/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [20]:
from keras.models import Model #Keras Functional API

from keras.layers import Input, Conv2D, UpSampling2D, Activation, LeakyReLU, Concatenate, BatchNormalization, Dropout

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization #InstanceNormalization layer is part of keras.contrib installed above

#from keras.initializers import RandomNormal

from keras.optimizers import Adam #import adam to specify settings

import numpy as np #import numpy numerical library

from glob import glob #glob is a function used for listing file path

from PIL import Image #Python Imaging library used here for simply opening image files and transforming input into bitmap format
#import cv2 #alternatively one can import the OpenCV library (famous for image processing)

#Visualization
import matplotlib.pyplot as plt #plot charts

from keras.utils import plot_model #visualization chart of the models

from tqdm import tqdm #progress bar

##Dataset 

https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Dataset

###Unzip and prepare working directory

In [22]:
#print working directory
%pwd

'/content/drive/My Drive/AI_for_Multimedia/Lezione12'

In [23]:
#print list files
%ls

apple2orange.zip


In [24]:
%cd '/content/drive/MyDrive/AI_for_Multimedia/Lezione12/'

/content/drive/MyDrive/AI_for_Multimedia/Lezione12


In [ ]:
#print list files
%ls

apple2orange.zip


In [ ]:
!unzip -q apple2orange.zip -d /content/